In [1]:
import chess
import chess.pgn
import numpy as np
from stockfish import Stockfish
import matplotlib.pyplot as plt


def load_game(pgn, index):
    i = 0
    while i < index:
        chess.pgn.skip_game(pgn)
        i += 1

    return chess.pgn.read_game(pgn)


def get_headers(game):
    game_times = game.headers['TimeControl'].split('+')
    time = int(float(game_times[0]) / 60)
    increment = '+' + game_times[1] if len(game_times) > 1 else ''
    return f"{game.headers['Date']} Time: {time}m{increment}\n" \
           f"{game.headers['White']} ({game.headers['WhiteElo']}) vs. " \
           f"{game.headers['Black']} ({game.headers['BlackElo']})\n" \
           f"{game.headers['Result']} {game.headers['Termination']}"


def mate_value(eval_value, is_white_move):
    if eval_value == 0:
        return -1 if not is_white_move else 1
    return -1 if eval_value < 0 else 1


def run(pgn_file, index):
    pgn = open(pgn_file)
    game = load_game(pgn, index)
    print(game)
    print(get_headers(game))
    stockfish = Stockfish()
    board = game.board()

    evaluations = []  # added - track evaluations for later plotting

    for move in game.mainline_moves():
        is_white_move = board.turn
        san = board.san(move)
        board.push(move)
        fen = board.fen()
        stockfish.set_fen_position(fen_position=fen)
        evaluation = stockfish.get_evaluation()
        eval_value = float(evaluation["value"]) / 1530
        print(f'{san} {eval_value} {evaluation["type"]}')

        evaluations.append(eval_value if evaluation["type"] != "mate" else mate_value(eval_value, is_white_move))

    evaluations = np.array(evaluations)

    fig = plt.figure()
    ax = fig.add_subplot(111)

    line, = ax.plot(evaluations, color='#e0e0e0', linewidth=0.5)

    ax.set_title(get_headers(game), fontdict={'fontsize': 9})
    ax.set_ylabel('Centipawn')
    ax.set_xlabel('Move')

    min_val, max_val = min(evaluations), max(evaluations)
    abs_max = max(abs(min_val), abs(max_val))
    plt.ylim(-abs_max, abs_max)

    fig.patch.set_facecolor('#F5F5E8')
    ax.set_facecolor('#F5F5FE')

    x = range(len(evaluations))
    plt.fill_between(x=x, y1=10, y2=evaluations, interpolate=True,
                     color='black', edgecolor=None)
    plt.fill_between(x=x, y1=-10, y2=evaluations,
                     interpolate=True, color='white', edgecolor=None)

    ax.margins(x=0)

    ax.axhline(0, color='#cccccc', linewidth=0.5, dashes=[6, 12])

    plt.show()
